In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

In [27]:
train = tf.data.Dataset.list_files('/content/drive/MyDrive/Medbikri/test/*.jpeg')

In [28]:
h, w = 512, 512
def normalize(input_image):
  input_image = tf.cast(input_image, tf.float32) / 255.0
  return input_image
def decode_img(img):
  img = tf.image.decode_jpeg(img)
  img = tf.image.resize(img, [h, w])
  return img
def process_path(file_path):
  img = normalize(decode_img(tf.io.read_file(file_path)))
  return img

In [29]:
train = train.map(process_path)

In [32]:
def predict(image, table_mask):
  table_mask = np.array(table_mask)
  image = np.array(image)
  contours, table_heirarchy = cv2.findContours(table_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
  tables=[]
  for c in contours : 
    if(cv2.contourArea(c) > 2000) : 
      tables.append(c)
  if(len(tables)==0):
    print("No table detected")
    return
  table_boundRect = [None]*len(tables)

  for i, c in enumerate(tables):
    polygon = cv2.approxPolyDP(c, 3, True)
    table_boundRect[i] = cv2.boundingRect(polygon)
  table_boundRect.sort()
  color = (0,0,255)
  thickness = 2

  for x,y,w,h in table_boundRect:
    cv2.rectangle(image, (x,y),(x+w,y+h), color, thickness)
  display(image)

In [33]:
def display(img):
  plt.figure(figsize=(15, 15))
  plt.title("image")
  plt.imshow(tf.keras.preprocessing.image.array_to_img(img))
  plt.axis('off')
  plt.show()

In [34]:
def create_mask(pred_mask1):
  pred_mask1 = tf.argmax(pred_mask1, axis=-1)
  pred_mask1 = pred_mask1[..., tf.newaxis]
  return pred_mask1[0]

In [35]:
def show_predictions(dataset=None):
  model = load_model('/content/drive/MyDrive/Medbikri/mdl1.h5')
  if dataset:
    for image in dataset:
      image = np.reshape(image, (1, 512, 512 , 3))
      pred_mask1= model.predict(image, verbose=1)
      table_mask= create_mask(pred_mask1)
      predict(tf.keras.preprocessing.image.array_to_img(image[0]), tf.keras.preprocessing.image.array_to_img(table_mask))

In [ ]:
show_predictions(train)